In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams
from langdetect import detect

import warnings
warnings.filterwarnings("ignore")

In [3]:
#load data into a dataframe
lddf = pd.read_csv('London-listings.csv', index_col = None, parse_dates = ['last_scraped', 'host_since'])
nydf = pd.read_csv('Newyork-listings.csv', index_col = None, parse_dates = ['last_scraped', 'host_since'])
tkdf = pd.read_csv('Tokyo-listings.csv', index_col = None, parse_dates = ['last_scraped', 'host_since'])

In [4]:
#take a look at dataset shape
print('dataset shape')
print('London:',lddf.shape)
print('New York:',nydf.shape)
print('Tokyo', tkdf.shape)

dataset shape
London: (86358, 106)
New York: (50378, 106)
Tokyo (15925, 106)


In [5]:
#take a look at what columns we have 
print(list(lddf.columns))

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price', '

In [6]:
#removing unnecessary data
lddf = lddf.drop(columns = ['id', 'listing_url', 'scrape_id', 'notes', 'access', 'house_rules', 
                        'thumbnail_url', 'medium_url','picture_url', 'xl_picture_url', 'host_url', 
                        'host_name', 'host_location','host_total_listings_count',
                        'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications',
                        'street', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 
                        'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 
                        'is_location_exact', 'square_feet', 'monthly_price', 'minimum_minimum_nights', 
                        'maximum_minimum_nights','minimum_maximum_nights', 'maximum_maximum_nights', 
                        'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 
                        'availability_30','availability_60', 'availability_90', 'availability_365', 
                        'calendar_last_scraped', 'number_of_reviews_ltm', 'first_review', 'last_review', 
                        'review_scores_accuracy', 'review_scores_checkin', 'review_scores_checkin', 
                        'review_scores_value', 'requires_license', 
                        'license', 'jurisdiction_names', 'is_business_travel_ready', 'cancellation_policy',
                        'require_guest_phone_verification', 'require_guest_profile_picture', 'calculated_host_listings_count',
                        'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                        'calculated_host_listings_count_shared_rooms', 'host_id', 'reviews_per_month', 'experiences_offered' ])

nydf = nydf.drop(columns = ['id', 'listing_url', 'scrape_id', 'notes', 'access', 'house_rules', 
                        'thumbnail_url', 'medium_url','picture_url', 'xl_picture_url', 'host_url', 
                        'host_name', 'host_location','host_total_listings_count',
                        'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications',
                        'street', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 
                        'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 
                        'is_location_exact', 'square_feet', 'monthly_price', 'minimum_minimum_nights', 
                        'maximum_minimum_nights','minimum_maximum_nights', 'maximum_maximum_nights', 
                        'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 
                        'availability_30','availability_60', 'availability_90', 'availability_365', 
                        'calendar_last_scraped', 'number_of_reviews_ltm', 'first_review', 'last_review', 
                        'review_scores_accuracy', 'review_scores_checkin', 'review_scores_checkin', 
                        'review_scores_value', 'requires_license', 
                        'license', 'jurisdiction_names', 'is_business_travel_ready', 'cancellation_policy',
                        'require_guest_phone_verification', 'require_guest_profile_picture', 'calculated_host_listings_count',
                        'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                        'calculated_host_listings_count_shared_rooms', 'host_id', 'reviews_per_month', 'experiences_offered' ])

tkdf = tkdf.drop(columns = ['id', 'listing_url', 'scrape_id', 'notes', 'access', 'house_rules', 
                        'thumbnail_url', 'medium_url','picture_url', 'xl_picture_url', 'host_url', 
                        'host_name', 'host_location', 'host_total_listings_count',
                        'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications',
                        'street', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 
                        'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 
                        'is_location_exact', 'square_feet', 'monthly_price', 'minimum_minimum_nights', 
                        'maximum_minimum_nights','minimum_maximum_nights', 'maximum_maximum_nights', 
                        'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 
                        'availability_30','availability_60', 'availability_90', 'availability_365', 
                        'calendar_last_scraped', 'number_of_reviews_ltm', 'first_review', 'last_review', 
                        'review_scores_accuracy', 'review_scores_checkin', 'review_scores_checkin', 
                        'review_scores_value', 'requires_license', 
                        'license', 'jurisdiction_names', 'is_business_travel_ready', 'cancellation_policy',
                        'require_guest_phone_verification', 'require_guest_profile_picture', 'calculated_host_listings_count',
                        'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                        'calculated_host_listings_count_shared_rooms', 'host_id', 'reviews_per_month', 'experiences_offered' ])


In [7]:
#use this function to modify data 
#in order to see whether certain information is provided
#if null or 0, return false, else true
def binary_change(item):  
    if pd.isnull(item) or item == '' or item == 0:
        return False
    else:
        return True

In [8]:
#modify variables for future use
for df in [lddf,nydf,tkdf]:
    df.price = df.price.replace(to_replace = r'\$',value = '', regex = True)
    df.price = df.price.replace(to_replace = r'\,',value = '', regex = True)
    df.price = df.price.astype(float)
    df.extra_people = df.extra_people.replace(to_replace = r'\$',value = '', regex = True)
    df.extra_people = df.extra_people.replace(to_replace = r'\,',value = '', regex = True)
    df.extra_people = df.extra_people.astype(float)
    df.host_response_rate = df.host_response_rate.replace(to_replace = r'\%', value ='', regex = True)
    df.host_response_rate = df.host_response_rate.astype(float)
    df.host_response_rate = df.host_response_rate.fillna(0)
    df.host_acceptance_rate = df.host_acceptance_rate.replace(to_replace = r'\%', value ='', regex = True)
    df.host_acceptance_rate = df.host_acceptance_rate.astype(float)
    df.host_acceptance_rate = df.host_acceptance_rate.fillna(0)
    df.review_scores_rating = df.review_scores_rating.fillna(0)
    df.review_scores_cleanliness = df.review_scores_cleanliness.fillna(0)
    df.review_scores_communication = df.review_scores_communication.fillna(0)
    df.review_scores_location = df.review_scores_location.fillna(0)
    df.transit = df.transit.apply(lambda x: binary_change(x))
    df.security_deposit = df.security_deposit.apply(lambda x: binary_change(x))
    df.cleaning_fee = df.cleaning_fee.apply(lambda x: binary_change(x))
    df.weekly_price = df.weekly_price.apply(lambda x: binary_change(x))
    df.host_response_time = df.host_response_time.fillna('None')
    df['time_as_host'] = df.last_scraped-df.host_since
    df.time_as_host = df.time_as_host.apply(lambda x: x.days)
    df = df.drop(columns = ['last_scraped', 'host_since'])


    df = df.dropna(inplace=True)
    

In [9]:
#remove outliers
lddf.describe()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,time_as_host
count,86358.000000,86358.000000,86348.000000,86358.000000,86226.000000,86216.000000,85416.000000,86358.000000,86358.000000,86358.000000,86358.000000,8.635800e+04,86358.000000,86358.000000,86358.000000,86358.000000,86358.000000,86348.000000
mean,57.378899,62.307476,25.805450,3.211793,1.312122,1.415399,1.756580,129.610725,1.610760,7.339644,4.474490,1.859559e+04,17.531277,67.690116,6.768591,7.050928,6.973702,1561.444133
std,47.256949,42.158875,134.219607,2.033954,0.611861,0.920860,1.291133,289.484146,1.368455,14.194697,19.151568,3.843374e+06,37.393948,41.921016,4.220715,4.345802,4.294565,831.586642
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,0.000000,0.000000,1.000000,2.000000,1.000000,1.000000,1.000000,47.000000,1.000000,0.000000,1.000000,3.100000e+01,1.000000,0.000000,0.000000,0.000000,0.000000,930.000000
50%,90.000000,86.000000,1.000000,2.000000,1.000000,1.000000,1.000000,83.000000,1.000000,0.000000,2.000000,1.125000e+03,4.000000,92.000000,9.000000,10.000000,9.000000,1598.000000
75%,100.000000,99.000000,5.000000,4.000000,1.500000,2.000000,2.000000,140.000000,2.000000,10.000000,3.000000,1.125000e+03,17.000000,98.000000,10.000000,10.000000,10.000000,2135.000000
max,100.000000,100.000000,1237.000000,32.000000,17.000000,24.000000,28.000000,19985.000000,46.000000,247.000000,1125.000000,1.000000e+09,773.000000,100.000000,10.000000,10.000000,10.000000,4155.000000


In [10]:
nydf.describe()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,time_as_host
count,50378.000000,50378.000000,50370.000000,50378.000000,50319.000000,50281.000000,49858.000000,50378.000000,50378.000000,50378.000000,50378.000000,5.037800e+04,50378.000000,50378.000000,50378.000000,50378.000000,50378.000000,50370.000000
mean,54.645738,58.701437,24.761148,2.859284,1.145114,1.183847,1.547334,164.437453,1.510620,13.878181,7.760411,8.665312e+04,24.765116,72.101870,7.123784,7.471734,7.364266,1694.601429
std,47.722585,42.930089,169.322231,1.886908,0.432479,0.771181,1.139372,449.524005,1.154335,24.117404,21.595536,1.353127e+07,48.853847,40.395311,4.034909,4.170266,4.111973,914.572226
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,0.000000,0.000000,1.000000,2.000000,1.000000,1.000000,1.000000,68.000000,1.000000,0.000000,1.000000,2.900000e+01,1.000000,70.000000,6.000000,8.000000,8.000000,1006.000000
50%,89.000000,80.000000,1.000000,2.000000,1.000000,1.000000,1.000000,105.000000,1.000000,0.000000,3.000000,1.119500e+03,5.000000,94.000000,9.000000,10.000000,10.000000,1723.000000
75%,100.000000,99.000000,2.000000,4.000000,1.000000,1.000000,2.000000,175.000000,2.000000,20.000000,5.000000,1.125000e+03,24.000000,99.000000,10.000000,10.000000,10.000000,2362.000000
max,100.000000,100.000000,2340.000000,22.000000,22.000000,22.000000,40.000000,25000.000000,16.000000,300.000000,1250.000000,2.147484e+09,746.000000,100.000000,10.000000,10.000000,10.000000,4252.000000


In [11]:
tkdf.describe()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,time_as_host
count,15925.000000,15925.000000,15914.000000,15925.000000,15925.000000,15905.000000,15898.000000,1.592500e+04,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15914.000000
mean,94.867441,94.628006,15.743622,4.465871,1.316923,1.365986,2.756699,2.419001e+04,2.380973,1480.265934,3.031962,665.472653,26.648226,77.401005,7.818085,8.080816,7.877865,903.971409
std,19.758392,17.862228,22.998848,3.192661,1.055243,1.373950,2.656215,7.336167e+04,1.963137,1588.739218,9.161638,480.289164,40.089704,35.240031,3.583118,3.659824,3.579132,633.459759
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
25%,100.000000,99.000000,3.000000,2.000000,1.000000,1.000000,1.000000,6.981000e+03,1.000000,0.000000,1.000000,179.000000,2.000000,83.000000,8.000000,9.000000,9.000000,380.000000
50%,100.000000,100.000000,8.000000,4.000000,1.000000,1.000000,2.000000,1.174500e+04,2.000000,1500.000000,1.000000,1124.000000,12.000000,93.000000,9.000000,10.000000,9.000000,717.000000
75%,100.000000,100.000000,19.000000,6.000000,1.500000,2.000000,3.000000,1.994500e+04,3.000000,2200.000000,2.000000,1125.000000,35.000000,97.000000,10.000000,10.000000,10.000000,1410.000000
max,100.000000,100.000000,196.000000,16.000000,50.000000,50.000000,56.000000,1.063959e+06,27.000000,30000.000000,365.000000,1125.000000,525.000000,100.000000,10.000000,10.000000,10.000000,3558.000000


In [12]:
##remove outliers
for df in [lddf, nydf, tkdf]:
    df = df[df.bedrooms < 7]
    df = df[df.host_listings_count < 30]
    df = df[df.minimum_nights <= 366]
    df = df[df.price < 1500]
    df = df[df.bathrooms < 8]
    df = df[df.beds > 0]
    df = df[df.bedrooms > 0]
    df = df[df.host_listings_count > 0]
    df = df[df.minimum_nights > 0]
    df = df[df.price > 0]
    df = df[df.bathrooms > 0]
    df = df[df.review_scores_rating  > 0]
    df = df[df.host_response_rate > 0]    
    df = df[df.host_acceptance_rate > 0]
    df = df[df.review_scores_cleanliness > 0]
    df = df[df.review_scores_communication > 0]
    df = df[df.review_scores_location > 0]
    df = df[df.number_of_reviews > 0]

    
    df = df.dropna(inplace=True)

In [30]:
#detect english 
def detecteng(text):
    try:
        return detect(text)
    except:
        return ''

In [41]:
for df in [lddf, nydf, tkdf]:
    df.dropna(inplace=True)

In [42]:
#remove all non-english
for df in [lddf, nydf, tkdf]:
    df['language'] = df.name.apply(lambda x: detecteng(x))
    df = df[df.language == 'en']
    df = df.drop(columns = ['language'])

In [43]:
def textpro(text):
    text = text.lower()    
    text = word_tokenize(text)   
    text = [WordNetLemmatizer().lemmatize(word) for word in text]   #lemmatize 
    stopword = set(stopwords.words('english'))
    text = [word for word in text if word not in stopword ] #delete stopwords
    text = [word for word if word.isalpha()]  #leave alphabets
    return text   

In [44]:
for df in [lddf, nydf, tkdf]:
    #pre-processing
    df.description = df.description.apply(lambda x: textpro(x))
    df.host_about = df.host_about.apply(lambda x: textpro(x))
    #bigram features
    df['bigram_description'] = df.description.apply(lambda x: list(ngrams(x, 2)))
    df['bigram_host_about'] = df.host_about.apply(lambda x: list(ngrams(x, 2)))

In [47]:
lddf.to_csv('new2_lddf.csv')

In [48]:
nydf.to_csv('new2_nydf.csv')

In [49]:
tkdf.to_csv('new2_tkdf.csv')